
# Phase Coherence & Vector Potentials — Interactive Lab

This notebook provides hands-on simulations for a graduate seminar on **Aharonov–Bohm phase**, **spin precession**, and **timing windows** (e.g., 3 ms vs 5 ms pulses).

**Modules:**
1. Phase accumulation under static vs accelerating (chirped) vector potentials.
2. Spin/Bloch dynamics with constant vs chirped frequency (resonance capture).
3. Pulse timing windows: comparing 3 ms and 5 ms spacing and their effect on phase-locking.
4. RF/IR drive timing generator (for exporting sequences).

> Notes:
> - Plots use `matplotlib` (no seaborn/styles).
> - Run cells top-to-bottom. If ipywidgets are unavailable, use the parameter cells to modify values and re-run.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from math import pi

# Matplotlib settings (simple, no styles, one plot per cell)
def newfig():
    plt.figure(figsize=(7,4))



## 1) Phase Accumulation: Static vs Chirped Vector Potentials

We model a phase increment:
\begin{equation}
\Delta \phi(t) = \frac{q}{\hbar} \int_0^t \oint A(\tau)\cdot dl \; d\tau \equiv k \int_0^t a(\tau)\, d\tau,
\end{equation}
where \(a(t)\) is a normalized potential proxy and \(k\) is a scaling constant (set to 1 here).

We compare:
- **Static** \(a(t)=a_0\)
- **Sinusoid** \(a(t)=\sin(2\pi f t)\)
- **Chirp** \(a(t)=\sin\left(2\pi (f_0 t + \frac{1}{2}\alpha t^2)\right)\) with sweep rate \(\alpha = d f/dt\).


In [ ]:

# Parameters
T = 0.2         # total time [s]
fs = 50000      # sample rate [Hz]
t = np.linspace(0, T, int(T*fs), endpoint=False)

a0 = 0.5        # static level
f_sin = 200.0   # sinusoid frequency [Hz]
f0 = 50.0       # chirp start [Hz]
alpha = 2000.0  # chirp rate [Hz/s]

# signals
a_static = np.full_like(t, a0)
a_sin = np.sin(2*np.pi*f_sin*t)
phase_chirp = 2*np.pi*(f0*t + 0.5*alpha*t**2)
a_chirp = np.sin(phase_chirp)

# integrate phase (cumulative sum)
dt = 1.0/fs
phi_static = np.cumsum(a_static)*dt
phi_sin    = np.cumsum(a_sin)*dt
phi_chirp  = np.cumsum(a_chirp)*dt

# Plot
newfig()
plt.plot(t, phi_static, label='Static')
plt.plot(t, phi_sin, label='Sinusoid')
plt.plot(t, phi_chirp, label='Chirp')
plt.xlabel('Time [s]'); plt.ylabel('Phase (arb.)'); plt.title('Accumulated Phase vs Time')
plt.legend()
plt.tight_layout()
plt.show()



## 2) Spin/Bloch Dynamics with Constant vs Chirped Frequency

We integrate a simple Bloch-like precession around the z-axis with transverse drive. The instantaneous angular frequency is:
\(\omega(t) = 2\pi f(t)\).

We compare a **constant** \(f=f_0\) vs a **chirped** \(f(t)=f_0 + \alpha t\). We visualize the transverse component magnitude \(|S_\perp(t)| = \sqrt{S_x^2 + S_y^2}\) as a proxy for resonance capture.


In [ ]:

# Parameters
T = 0.1
fs = 100000
t = np.linspace(0, T, int(T*fs), endpoint=False)
dt = 1.0/fs

f0 = 1000.0     # base frequency [Hz]
alpha = 20000.0 # chirp rate [Hz/s]
drive_amp = 1.0 # normalized

def simulate(f_mode='constant'):
    Sx, Sy, Sz = 1.0, 0.0, 0.0  # initial spin vector
    Sx_hist, Sy_hist, Sz_hist = [], [], []
    phase = 0.0
    for ti in t:
        if f_mode == 'constant':
            f = f0
        else:
            f = f0 + alpha*ti
        omega = 2*np.pi*f
        # simple rotation around z with small transverse drive modulation
        phase += omega*dt
        # Precession
        Sx_new =  np.cos(omega*dt)*Sx - np.sin(omega*dt)*Sy
        Sy_new =  np.sin(omega*dt)*Sx + np.cos(omega*dt)*Sy
        # small transverse drive: nudges towards x with amplitude*dt
        Sx, Sy, Sz = Sx_new + drive_amp*dt*np.cos(phase), Sy_new + drive_amp*dt*np.sin(phase), Sz
        # normalize occasionally
        if int(ti*fs) % 1000 == 0:
            norm = max(1e-9, np.sqrt(Sx*Sx + Sy*Sy + Sz*Sz))
            Sx, Sy, Sz = Sx/norm, Sy/norm, Sz/norm
        Sx_hist.append(Sx); Sy_hist.append(Sy); Sz_hist.append(Sz)
    return np.array(Sx_hist), np.array(Sy_hist), np.array(Sz_hist)

Sx_c, Sy_c, Sz_c = simulate('constant')
Sx_ch, Sy_ch, Sz_ch = simulate('chirp')

newfig()
plt.plot(t, np.sqrt(Sx_c**2 + Sy_c**2), label='Constant f0')
plt.plot(t, np.sqrt(Sx_ch**2 + Sy_ch**2), label='Chirp (resonance capture)')
plt.xlabel('Time [s]'); plt.ylabel('|S_perp(t)|'); plt.title('Spin Transverse Component vs Time')
plt.legend()
plt.tight_layout()
plt.show()



## 3) Pulse Timing Windows: 3 ms vs 5 ms

We compare trains of **square pulses** with ON duration \(d_{on}\) and OFF intervals \(T_{off}=\{3,5\}\) ms. We drive a damped oscillator:
\(\ddot{x} + 2\zeta\omega_0 \dot{x} + \omega_0^2 x = u(t)\),
and measure the **phase alignment** between \(u(t)\) and \(x(t)\) over time.


In [ ]:

# Parameters
fs = 20000
T = 0.5
t = np.linspace(0, T, int(T*fs), endpoint=False)
dt = 1.0/fs

f0 = 40.0            # natural frequency [Hz]
omega0 = 2*np.pi*f0
zeta = 0.1           # damping
don = 0.002          # 2 ms ON pulse

def pulse_train(toff_ms):
    u = np.zeros_like(t)
    toff = toff_ms/1000.0
    t_next = 0.0
    while t_next < T:
        idx_on = (t >= t_next) & (t < t_next + don)
        u[idx_on] = 1.0
        t_next += don + toff
    return u

def drive_system(u):
    x = 0.0; v = 0.0
    x_hist = []
    for ui in u:
        # x'' = u - 2 zeta omega0 v - omega0^2 x
        a = ui - 2*zeta*omega0*v - (omega0**2)*x
        v += a*dt
        x += v*dt
        x_hist.append(x)
    return np.array(x_hist)

u3 = pulse_train(3.0)
u5 = pulse_train(5.0)
x3 = drive_system(u3)
x5 = drive_system(u5)

def phase_alignment(u, x):
    # compute normalized cross-correlation at zero lag
    u_z = (u - u.mean())
    x_z = (x - x.mean())
    num = np.sum(u_z * x_z)
    den = np.sqrt(np.sum(u_z**2)*np.sum(x_z**2) + 1e-12)
    return num/den

newfig()
plt.plot(t, u3, label='Input pulses (3 ms OFF)')
plt.plot(t, x3, label='Response x(t)')
plt.xlabel('Time [s]'); plt.ylabel('Amplitude'); plt.title('3 ms Window')
plt.legend(); plt.tight_layout(); plt.show()

newfig()
plt.plot(t, u5, label='Input pulses (5 ms OFF)')
plt.plot(t, x5, label='Response x(t)')
plt.xlabel('Time [s]'); plt.ylabel('Amplitude'); plt.title('5 ms Window')
plt.legend(); plt.tight_layout(); plt.show()

print("Phase alignment (3 ms):", phase_alignment(u3, x3))
print("Phase alignment (5 ms):", phase_alignment(u5, x5))



## 4) RF/IR Drive Timing Generator

Creates arrays of timestamps for IR and RF bursts following a **randomized + descending** rhythm pattern, suitable for exporting to a microcontroller.


In [ ]:

rng = np.random.default_rng(123)

def gen_rhythm():
    timeline = []
    t0 = 0.0
    # four randomized pulses
    for _ in range(4):
        on = rng.integers(6, 30)/1000.0
        timeline.append((t0, t0+on, 'RAND'))
        t0 += on + 0.003
    # descending sequence (off delays with fixed 3 ms on)
    offs = [24,22,20,18,16,14,12,10,8,6]
    for off in offs:
        on = 0.003
        timeline.append((t0, t0+on, 'PHASE'))
        t0 += on + off/1000.0
    # rest
    t0 += rng.integers(100, 501)/1000.0
    return timeline, t0

timeline, total_T = gen_rhythm()
print("Total sequence length [s]:", total_T)
print("First 10 events:")
for i, (ts, te, tag) in enumerate(timeline[:10]):
    print(i, tag, f"{ts:.6f} -> {te:.6f}")

# Optional: visualize as a simple raster
newfig()
for (ts, te, tag) in timeline:
    plt.hlines(1 if tag=='RAND' else 0.6, ts, te, linewidth=4)
plt.ylim(0, 1.2); plt.xlabel('Time [s]'); plt.ylabel('Burst type')
plt.title('RF/IR Rhythm Timeline (RAND vs PHASE)')
plt.tight_layout(); plt.show()
